In [2]:
import pandas as pd
from haversine import haversine as hvs, Unit
import numpy as np

# corregir puntos de stops

In [6]:
def splitPoint(row):
    row['latitude'] = row['WKT'].split()[2].strip()[:-1]
    row['longitude'] = row['WKT'].split()[1].strip()[1:]
    return row

data_busstops = pd.read_csv('Puntos por Vehiculos- busstops_ida.csv.csv')
# data_busstops = data_busstops.apply(splitPoint, axis=1)
data_busstops = data_busstops.sort_values(by='number').reset_index(drop=True)
data_busstops = data_busstops.drop(['number', 'WKT'], axis=1) # escoger los paraderos de ida
# data_busstops['id'] = range(36, len(data_busstops)+36)
data_busstops['id'] = data_busstops['id'].apply(lambda id: str(int(id)))
data_busstops['navigation'] = data_busstops['navigation'].values.astype(np.int64)
data_busstops['radio'] = data_busstops['radio'].values.astype(np.int64)

# data_busstops.append(pd.Series([0, 'start', 0, ]))
print(data_busstops.to_string())

     id                       name  navigation   latitude  longitude  radio
0     0                     inicio         159 -13.554978 -71.856025     30
1    36                primer stop         304 -13.554812 -71.857300     18
2    37               segundo stop         281 -13.553294 -71.861944     13
3    38                   Ccollana         283 -13.552296 -71.867299      9
4    39            Puente Huaccoto         283 -13.551592 -71.870211     10
5    40                      Kayra         289 -13.550619 -71.873963     10
6    41                   Forestal         286 -13.550020 -71.876104      8
7    42                Grifo Tapia         284 -13.549259 -71.878642     11
8    43                      Posta         283 -13.548860 -71.880031     15
9    44                  Romeritos         284 -13.548657 -71.880933      7
10   45                    Tercero         286 -13.548093 -71.882989      7
11   46                      Medio         286 -13.547786 -71.884096      7
12   47     

In [39]:
# df_stops_ida = pd.read_csv('busstops_ida.csv')
# data_busstops['radio'] = df_stops_ida['radio']
# data_busstops

In [7]:
data_busstops.to_csv('busstops_ida.csv')

# Extraer en CSV

In [11]:
# leer dataset monitoring
dt = pd.read_csv('_Process_monitoring__202303251137.csv')
dt['vehicle_id_id'].unique()


# diseño del df: creacion data_time y latitude_longitude
dt['date_time'] = dt['date']  + ' ' + dt['time']
dt['latitude'] = dt.apply(lambda row: (row['latitude'], row['longitude']), axis=1)
dt.drop(['longitude'], inplace=True, axis=1)
dt = dt.rename(columns={'latitude':'latitude_longitude', 'vehicle_id_id':'vehicle_id'}, inplace=False)
dt = dt.drop_duplicates(subset=['latitude_longitude']) # (65155, 7) (64702, 7)
dt = dt.sort_values(by='date_time')

# convertir de object a datatime
dt['date_time'] = pd.to_datetime(dt['date_time'], format='%Y-%m-%d %H:%M:%S')
print(dt.shape)
dt

(1671363, 9)


,id,date,time,lap,latitude_longitude,velocity,navigation,vehicle_id,date_time
29756,13172252,2023-03-01,00:01:34,5,"(-13.5513563, -71.9902231)",1,140,SJ40,2023-03-01 00:01:34
29311,13172234,2023-03-01,00:01:49,0,"(-13.5506013, -71.8786173)",2,185,SJ10,2023-03-01 00:01:49
29780,13172253,2023-03-01,00:03:25,0,"(-13.5506231, -71.8786667)",3,206,SJ10,2023-03-01 00:03:25
31234,13172309,2023-03-01,00:05:13,5,"(-13.5383472, -71.9807985)",0,0,SJ08,2023-03-01 00:05:13
29831,13172255,2023-03-01,00:05:30,0,"(-13.55065, -71.8785454)",3,113,SJ10,2023-03-01 00:05:30
...,...,...,...,...,...,...,...,...,...
1585989,17270532,2023-03-25,11:37:16,2,"(-13.5504177, -71.9870624)",31,230,SJ03,2023-03-25 11:37:16
1585942,17270531,2023-03-25,11:37:16,3,"(-13.550582, -71.8741605)",16,285,SJ20,2023-03-25 11:37:16
1586202,17270537,2023-03-25,11:37:17,3,"(-13.5217018, -71.9646513)",19,289,SJ32,2023-03-25 11:37:17
1586071,17270534,2023-03-25,11:37:17,2,"(-13.5405389, -71.9812735)",38,13,SJ04,2023-03-25 11:37:17


In [12]:
# FUNCION: ver si un punto paso por un stop
def itPassed(point, stop):
    dnav = abs(point['navigation'] - stop['navigation'])
    dis = round(hvs(point['latitude_longitude'], stop['latitude_longitude'], unit=Unit.METERS), 2)
    return (dnav<=45 and dis<=stop['radio'])

# FUNCION: filtrar los puntos de ida
def idaPoints(df):
    end1_stop = {'latitude_longitude':(-13.5513636,-71.988304), 'navigation':75, 'radio':50}
    end2_stop = {'latitude_longitude':(-13.5511395,-71.986925), 'navigation':206, 'radio':50}
    for i, row in df.iterrows():
        if ( itPassed(row, end1_stop) or itPassed(row, end2_stop)):
            break
    return df.head(i+1)

In [13]:
dt['vehicle_id'].unique()

array(['SJ40', 'SJ10', 'SJ08', 'SJ37', 'SJ27', 'SJ28', 'SJ03', 'SJ05',
       'SJ14', 'SJ31', 'SJ20', 'SJ16', 'SJ41', 'SJ35', 'SJ34', 'SJ39',
       'SJ04', 'SJ01', 'SJ06', 'SJ42', 'SJ36', 'SJ23', 'SJ24', 'SJ33',
       'SJ38', 'SJ29', 'SJ21', 'SJ32', 'SJ25', 'SJ30', 'SJ09', 'SJ07',
       'SJ26', 'SJ22', 'SJ15', 'SJ13', 'SJ12', 'SJ18', 'SJ17', 'SJ43',
       'SJ19', 'SJ11', 'SJ02'], dtype=object)

In [14]:
def quierodatos(veh, fecha):
    # parametros de extraccion
    # veh = 'SJ23'
    # fecha = '2023-03-20'
    dt_veh = dt[(dt['vehicle_id'] == veh) & (dt['date'] == fecha)]
    laps = sorted(list(dt_veh['lap'].unique())[1:-1])
    # laps = [2] 
    if len(laps)== 0:
        print(f'No hay nada {veh}-{fecha}')
        return False
     
    print(sorted(list(dt_veh['lap'].unique())), '->', laps)

    dt_vehlap = pd.DataFrame()
    for lap in laps:
        tmp =  idaPoints(dt_veh[dt_veh['lap']==lap].reset_index(drop=True))
        dt_vehlap = pd.concat([dt_vehlap, tmp])
        print(f"{veh} {lap} -> {tmp.shape[0]}")

    # print(dt_vehlap)
    if len(dt_vehlap['lap'].unique()) != 1:
        name = 'monitoring_veh/' + str(veh) + '-' + fecha + '.csv'
        dt_vehlap.to_csv(name)
        print(f'print data ({veh}, {laps}, {dt_vehlap.shape[0]}) -> {name}')
    else:
        name = 'monitoring_veh_lap/' + str(veh) + '-lap' + str(dt_vehlap['lap'].unique()[0]) + '.csv'
        dt_vehlap.to_csv(name)
        print(f'print data ({veh}, {laps}, {dt_vehlap.shape[0]}) -> {name}')

    return dt_vehlap

In [16]:
# quierodatos('SJ30', '2023-03-20')
# quierodatos('SJ39', '2023-03-17')
print(quierodatos('SJ01', '2023-03-25').to_string())
# quierodatos('SJ28', '2023-03-01')
# quierodatos('SJ22', '2023-03-25')

[0, 1, 2, 6] -> [1, 6]
SJ01 1 -> 212
SJ01 6 -> 1
print data (SJ01, [1, 6], 213) -> monitoring_veh/SJ01-2023-03-25.csv
           id        date      time  lap          latitude_longitude  velocity  navigation vehicle_id           date_time
0    17222045  2023-03-25  06:58:32    1  (-13.5555891, -71.8562918)        21         304       SJ01 2023-03-25 06:58:32
1    17222067  2023-03-25  06:58:44    1  (-13.5550823, -71.8570471)        32         304       SJ01 2023-03-25 06:58:44
2    17222119  2023-03-25  06:59:06    1  (-13.5546515, -71.8576435)        20         303       SJ01 2023-03-25 06:59:06
3    17222173  2023-03-25  06:59:30    1  (-13.5542364, -71.8582292)        22         302       SJ01 2023-03-25 06:59:30
4    17222202  2023-03-25  06:59:42    1  (-13.5538248, -71.8590588)        35         290       SJ01 2023-03-25 06:59:42
5    17222234  2023-03-25  06:59:54    1  (-13.5535063, -71.8603615)        47         278       SJ01 2023-03-25 06:59:54
6    17222257  2023-03-25  0